### 2) Sample city selection_Have Customers Narrow Their Travel Searches Based on Temperature

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import matplotlib as plt
# Import the API key.
from config2 import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("../WeatherAnalysis/data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yar-Sale,RU,2021-08-26 22:23:53,66.8333,70.8333,40.55,93,100,27.92,light rain
1,1,Punta Arenas,CL,2021-08-26 22:22:04,-53.1500,-70.9167,39.31,75,20,2.30,few clouds
2,2,Bluff,NZ,2021-08-26 22:23:54,-46.6000,168.3333,43.81,74,97,11.32,overcast clouds
3,3,Kavieng,PG,2021-08-26 22:23:55,-2.5744,150.7967,83.41,72,74,10.71,broken clouds
4,4,Port Alfred,ZA,2021-08-26 22:23:55,-33.5906,26.8910,62.89,91,10,13.78,clear sky


In [3]:
# Get the data types
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Date                    object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

In [4]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 55
What is the maximum temperature you would like for your trip? 95


In [5]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                           (city_data_df["Max Temp"] >= min_temp) ]
        
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Kavieng,PG,2021-08-26 22:23:55,-2.5744,150.7967,83.41,72,74,10.71,broken clouds
4,4,Port Alfred,ZA,2021-08-26 22:23:55,-33.5906,26.8910,62.89,91,10,13.78,clear sky
6,6,Puerto Escondido,MX,2021-08-26 22:23:56,15.8500,-97.0667,81.25,79,100,24.40,overcast clouds
9,9,Butaritari,KI,2021-08-26 22:23:57,3.0707,172.7902,81.64,76,99,14.14,light rain
10,10,Puerto Ayora,EC,2021-08-26 22:23:57,-0.7393,-90.3518,75.16,81,76,8.99,broken clouds
11,11,Rikitea,PF,2021-08-26 22:23:58,-23.1203,-134.9692,71.85,73,11,10.60,few clouds
12,12,Hwange,ZW,2021-08-26 22:23:58,-18.3693,26.5019,67.93,35,0,3.02,clear sky
14,14,Tam Ky,VN,2021-08-26 22:19:07,15.5667,108.4833,78.78,95,100,6.78,overcast clouds
16,16,Krasnoselkup,RU,2021-08-26 22:24:00,65.7000,82.4667,61.18,93,99,16.02,overcast clouds
18,18,Atuona,PF,2021-08-26 22:24:00,-9.8000,-139.0333,77.58,76,1,18.05,clear sky


In [6]:
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City_ID","City", "Country", "Max Temp","Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,3,Kavieng,PG,83.41,broken clouds,-2.5744,150.7967,
4,4,Port Alfred,ZA,62.89,clear sky,-33.5906,26.8910,
6,6,Puerto Escondido,MX,81.25,overcast clouds,15.8500,-97.0667,
9,9,Butaritari,KI,81.64,light rain,3.0707,172.7902,
10,10,Puerto Ayora,EC,75.16,broken clouds,-0.7393,-90.3518,
11,11,Rikitea,PF,71.85,few clouds,-23.1203,-134.9692,
12,12,Hwange,ZW,67.93,clear sky,-18.3693,26.5019,
14,14,Tam Ky,VN,78.78,overcast clouds,15.5667,108.4833,
16,16,Krasnoselkup,RU,61.18,overcast clouds,65.7000,82.4667,
18,18,Atuona,PF,77.58,clear sky,-9.8000,-139.0333,


In [8]:
# Dependencies and Setup
import requests
import gmaps
import pprint

# Import API key
from config2 import g_key

In [9]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

In [10]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
       hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
       print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [11]:
hotel_df.head(10)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,3,Kavieng,PG,83.41,broken clouds,-2.5744,150.7967,Nusa Island Retreat
4,4,Port Alfred,ZA,62.89,clear sky,-33.5906,26.8910,The Halyards Hotel
6,6,Puerto Escondido,MX,81.25,overcast clouds,15.8500,-97.0667,Aldea del Bazar Hotel and Spa
9,9,Butaritari,KI,81.64,light rain,3.0707,172.7902,Isles Sunset Lodge
10,10,Puerto Ayora,EC,75.16,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
11,11,Rikitea,PF,71.85,few clouds,-23.1203,-134.9692,People ThankYou
12,12,Hwange,ZW,67.93,clear sky,-18.3693,26.5019,Hwange Colliery Guest House
14,14,Tam Ky,VN,78.78,overcast clouds,15.5667,108.4833,Khách Sạn Đông Á
16,16,Krasnoselkup,RU,61.18,overcast clouds,65.7000,82.4667,Krasnosel'kupnefteprodukt
18,18,Atuona,PF,77.58,clear sky,-9.8000,-139.0333,Villa Enata


In [12]:
hotel_df.dtypes

City_ID                  int64
City                    object
Country                 object
Max Temp               float64
Current Description     object
Lat                    float64
Lng                    float64
Hotel Name              object
dtype: object

In [13]:
hotel_df = hotel_df.set_index('City_ID')

In [14]:
# Create the output file (CSV).
output_data_file = "data/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index ="City ID")

In [15]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

In [16]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [20]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]

#Find locations around New York.
new_york_coordinates = (40.75, -74.00)
fig = gmaps.figure( center=new_york_coordinates, 
                   zoom_level=3, 
                   layout={'width': '800px', 'height': '480px'})

marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='480px', width='800px'))